Кобзарь О.С. Хабибуллин Р.А. 31.07.2019

# Управление расчетом VBA

1. Добавление данных из шахматки.
2. Сам расчет и хранение данных.
3. Генерация общего файлика с проадаптированной скважиной во времени

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

from datetime import date
import xlwings as  xw
pd.set_option('precision', 3)

# 1.  Добавление недостающих данных из шахматки

Подготовленные данные со станции управления

In [2]:
cs_data_resampled = pd.read_csv("resampled_small_data.csv", index_col = "2")
cs_data_resampled.index = pd.to_datetime(cs_data_resampled.index)
cs_data_resampled.head()

,Активная мощность,Выходная частота ПЧ,Давление на приеме насоса (пласт. жидкость),Загрузка двигателя,Коэффициент мощности,Массовый дебит жидкости,Массовый дебит нефти,Напряжение на выходе ТМПН,Объемный дебит воды,Объемный дебит газа,Объемный дебит жидкости,Объемный дебит нефти,Полная мощность,Процент обводненности,Температура на приеме насоса (пласт. жидкость)
2,,,,,,,,,,,,,,,
2019-02-01,81.209,NaN,2.992,55.828,0.708,NaN,NaN,1546.854,NaN,NaN,NaN,NaN,113.975,NaN,NaN
2019-02-02,81.297,NaN,2.995,55.858,0.708,115.29,93.190,1546.849,22.34,2127.27,122.000,99.66,113.995,18.65,23.975
2019-02-03,81.489,NaN,2.995,56.005,0.709,114.01,91.320,1546.708,22.93,2062.95,120.640,97.71,114.139,25.12,23.815
2019-02-04,81.223,19.0,2.996,55.842,0.707,117.48,94.415,1541.794,23.32,2160.85,124.315,101.00,113.830,22.47,21.500
2019-02-05,81.154,NaN,2.991,55.794,0.708,113.50,91.580,1546.753,22.16,2244.05,120.100,97.94,113.833,18.09,24.005


Данные из шахматки

In [3]:
chess_data = pd.read_excel("Скв. 1354 шахматка.xls", skiprows = 2)
chess_data.index = pd.to_datetime(chess_data['Unnamed: 0'])
chess_data.head()

,Unnamed: 0,Тип,Нсп,ТМ,Дисп,OIS,Qж ТМ,Qн ТМ,Обв ТМ,Рбуф ТМ,...,T на приёме нас.,Темп. ж. ТМ,Тнасос ТМ,I A ТМ,I C ТМ,I В ТМ,R изол ТМ,R изоляц.,U раб.,Unnamed: 39
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2019-03-31,2019-03-31,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,65,21,0,42.15,43.28,41.88,9999.0,9999.0,NaN,29.256
2019-03-30,2019-03-30,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,65,21,0,42.15,43.28,41.88,9999.0,9999.0,NaN,NaN
2019-03-29,2019-03-29,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,65,21,0,42.15,43.28,41.88,9999.0,9999.0,NaN,NaN
2019-03-28,2019-03-28,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,65,21,0,42.15,43.28,41.88,9999.0,9999.0,NaN,NaN
2019-03-27,2019-03-27,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,65,21,0,42.15,43.28,41.88,9999.0,9999.0,NaN,NaN


Объединение данных для получения всех параметров, необходимых для расчета. Расчет ГФ

In [4]:
input_data = chess_data.join(cs_data_resampled)
input_data["ГФ"] = input_data[" Объемный дебит газа"] / input_data[" Объемный дебит нефти"]
input_data.to_csv("input_data.csv")
#input_data.iloc[-40]
input_data.head()

,Unnamed: 0,Тип,Нсп,ТМ,Дисп,OIS,Qж ТМ,Qн ТМ,Обв ТМ,Рбуф ТМ,...,Массовый дебит нефти,Напряжение на выходе ТМПН,Объемный дебит воды,Объемный дебит газа,Объемный дебит жидкости,Объемный дебит нефти,Полная мощность,Процент обводненности,Температура на приеме насоса (пласт. жидкость),ГФ
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
2019-03-31,2019-03-31,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-30,2019-03-30,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-29,2019-03-29,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-28,2019-03-28,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-03-27,2019-03-27,ЭЦН5А-320-1500,830 м,Запуск,NaN,раб.,123,90.67,21.75,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 2. Массовый полуавтоматизированный расчет

Создание интерфейсных функций в питоне, способных вызывать функции в UniflocVBA

In [5]:
book = xw.Book("UF7_калькулятор_ЭЦН _1354_с_СУ.xlsm")

f_write_on_sheet = book.macro("write_on_sheet")
f_get_data = book.macro("get_data")

Необходимые функции для подготовки данных и передачи их в UniflocVBA

In [6]:
class well_state_in_one_day():  # структура для хранения параметров состояния скважины в конкретный момент времени
    def __init(self, Ql_scm3day = None,
               wc_perc = None,
               Rp_m3m3 = None, 
              P_intake_in_atma = None,
              Pline_atma = None,
              Pbuf_atma = None,
              Pwf_atma = None,
              Pcas_atma = None,
              I_motor_nom_A_in = None,
              Power_motor_nom_kWt = None,
              I_motor_A_in = None,
               cos_phi_ = None,
               load_motor_in_d = None,
               U_V_ = None,
               act_power_cs_kwt_in = None):
        
        self.Ql_scm3day = Ql_scm3day
        self.wc_perc = wc_perc
        self.Rp_m3m3 = Rp_m3m3
        
        self.P_intake_in_atma = P_intake_in_atma
        self.Pline_atma = Pline_atma
        self.Pbuf_atma = Pbuf_atma
        self.Pwf_atma = Pwf_atma
        self.Pcas_atma = Pcas_atma
        
        self.I_motor_nom_A_in = I_motor_nom_A_in
        self.Power_motor_nom_kWt = Power_motor_nom_kWt
        self.I_motor_A_in = I_motor_A_in
        self.cos_phi_ = cos_phi_
        self.load_motor_in_d = load_motor_in_d
        self.U_V_= U_V_
        self.act_power_cs_kwt_in = act_power_cs_kwt_in
        
def transfer_data_from_df_to_obj(df, obj): # заполнение структуры данными из DataFrame, содержающего одну строку для нужного времени
    obj.Ql_scm3day = df[" Объемный дебит жидкости"][0]
    obj.wc_perc = df[" Процент обводненности"][0]
    obj.Rp_m3m3 = df["ГФ"][0]

    obj.P_intake_in_atma = df["Рприем"][0]
    obj.Pline_atma = df["Рлин ТМ"][0]
    obj.Pbuf_atma = df["Рбуф"][0]
    obj.Pwf_atma = df["Pзаб(Pпр)"][0]
    obj.Pcas_atma = df["Рзатр"][0]

    obj.I_motor_nom_A_in = 64
    obj.Power_motor_nom_kWt = 140
    obj.I_motor_A_in = df["I"][0]
    obj.cos_phi_ = df[" Коэффициент мощности"][0]
    obj.load_motor_in_d = df[" Загрузка двигателя"][0] / 100
    obj.U_V_= df[" Напряжение на выходе ТМПН"][0]
    obj.act_power_cs_kwt_in = df[" Активная мощность"][0]
    return obj

def write_float_data_on_list(func_xw, well_state_in_one_day_obj):  # запуск процедуры в VBA, записывающей данные на листи
    func_xw(float(well_state_in_one_day_obj.Ql_scm3day),
            float(well_state_in_one_day_obj.wc_perc),
            float(well_state_in_one_day_obj.Rp_m3m3),
            float(well_state_in_one_day_obj.P_intake_in_atma),
            float(well_state_in_one_day_obj.Pline_atma),
            float(well_state_in_one_day_obj.Pbuf_atma),
            float(well_state_in_one_day_obj.Pwf_atma),
            float(well_state_in_one_day_obj.Pcas_atma),
            float(well_state_in_one_day_obj.I_motor_nom_A_in),
            float(well_state_in_one_day_obj.Power_motor_nom_kWt),
            float(well_state_in_one_day_obj.I_motor_A_in),
            float(well_state_in_one_day_obj.cos_phi_),
            float(well_state_in_one_day_obj.load_motor_in_d),
            float(well_state_in_one_day_obj.U_V_ ),
            float(well_state_in_one_day_obj.act_power_cs_kwt_in))
    
def get_state_by_timestamp(data, timestamp):  # заполнение структуры данными в нужный момент времени 
    one_day_input_param = data[data.index == timestamp]
    empty_state = well_state_in_one_day()
    filled_state = transfer_data_from_df_to_obj(one_day_input_param, empty_state)
    return filled_state

Время, для которого ведется расчет

In [16]:
year_calculated = 2019
month_calculated = 2
day_calculated = 15
hour_calculated = 0

Собственно сам расчет, в котором:

1. Извлекаются данные в требуемый момент времени из хранилища исходных данных
2. Заполняется структура всеми необходимыми данными
3. С помощью xlwing вызывается функия VBA с параметрами, переданными через структуру, которая записывает данные на лист
4. Вызвается с помощью xlwing функция, расчитывающая параметры и возвращающая их обратно в питон
5. Рассчитанные параметры записываются в отдельный .csv

Workflow:
1. Выбрать день и записать параметры на лист с помощью функций
2. Проадаптировать скважину
3. Еще раз запустить функции для сохранения
4. Перейти к следующему моменту времени

In [17]:
date_from = pd.to_datetime(pd.DataFrame({'year': [year_calculated],
                           'month': [month_calculated],
                           'day': [day_calculated],
                            'hour': [hour_calculated]}))
day_timestamp = date_from.iloc[0]
day_in_string = str(day_calculated) + "." + str(month_calculated)+ "." + str(year_calculated)
state_data_in_one_day_obj = get_state_by_timestamp(input_data, day_timestamp)

write_float_data_on_list(f_write_on_sheet, state_data_in_one_day_obj)

calculated_data = f_get_data()

calculated_data_in_dict = {}
calculated_data_in_dict.update({'Дата': [day_in_string]})
values = calculated_data[0]
names = calculated_data[1]
for name, value in zip(names, values):
    calculated_data_in_dict.update({name: [value]})
    
calculated_data_in_df = pd.DataFrame(calculated_data_in_dict)
calculated_data_in_df.to_csv(str(day_calculated) + ".csv")

result_data = pd.read_csv(str(day_calculated) +".csv")
result_data.iloc[0]

Unnamed: 0                                    0
Дата                                  15.2.2019
PVT.Qliq_scm3day                            124
PVT.wc_perc                                18.2
PVT.Rp_m3m3                                25.8
Pwf_                                          0
tbh_                                         16
ESP.ESPpump.freq_Hz                          38
ESP.Power_nom_W                         1.4e+05
ESP.cos_phi                               0.708
ESP.U_V                                1.55e+03
ESP.motor_eff_fr                           0.89
ESP.ESPPressureDegradation                 0.15
ESP.ESPpump.RateDegradation                0.05
ESP.power_degr_fr                           0.1
Разделитель данных                            0
well.Pline_atma                            21.2
well.Pwf_atma                              39.7
well.Tline_C                               21.5
well.Twh_C                                 21.5
well.Tbh_C                              

# 3. Генерация общего файла с расчетами и предварительный просмотр

Объединение множества файликов с расчетом для одного момента времени

In [9]:
result_data = pd.read_csv("28.csv")
for i in range(27, 1, -1):
    calc_data = pd.read_csv(str(i) + ".csv")
    result_data = result_data.append(calc_data, ignore_index=True)
result_data.to_csv("result_data.csv")
result_data.head()    

,Unnamed: 0,Дата,PVT.Qliq_scm3day,PVT.wc_perc,PVT.Rp_m3m3,Pwf_,tbh_,ESP.ESPpump.freq_Hz,ESP.Power_nom_W,ESP.cos_phi,...,well.ESP.ESPpump.PowerFluid_Wt / k,well.ESP.cable_Power_W / k,well.ESP.Power_CS_calc_W / k,well.ESP.cable_dPower_W / k,well.ESP.dPower_GasSep_W / k,well.ESP.dPower_protector_W / k,well.ESP.dPower_transform_W / k,well.ESP.I_A,well.ESP.ESPpump.PowerESP_Wt / k,Unnamed: 37
0,0,28,123.533,19.480,23.876,0.0,16.0,38.0,140000.0,0.720,...,11.537,88.948,85.081,4.078,0.5,0.4,2.751,43.908,74.634,0.0
1,0,27,124.194,18.190,25.769,0.0,16.0,38.0,140000.0,0.708,...,11.371,85.653,81.929,3.869,0.5,0.4,2.649,43.140,71.887,0.0
2,0,28,123.637,21.785,20.968,0.0,16.0,38.0,140000.0,0.711,...,11.397,91.929,87.933,4.412,0.5,0.4,2.843,45.967,76.990,0.0
3,0,28,127.462,19.430,20.423,0.0,16.0,38.0,140000.0,0.710,...,11.783,89.699,85.800,4.213,0.5,0.4,2.774,44.932,75.182,0.0
4,0,28,133.575,21.720,16.944,0.0,16.0,38.0,140000.0,0.710,...,12.590,93.013,88.970,4.517,0.5,0.4,2.877,46.545,77.861,0.0


Список параметров рассчитанных и некоторых входных

In [10]:
needed_param_list = []
for namber, name in enumerate(result_data.columns):
    print(str(namber) + ' ' + name)
    needed_param_list.append(name)

0 Unnamed: 0
1 Дата
2 PVT.Qliq_scm3day
3 PVT.wc_perc
4 PVT.Rp_m3m3
5 Pwf_
6 tbh_
7 ESP.ESPpump.freq_Hz
8 ESP.Power_nom_W
9 ESP.cos_phi
10 ESP.U_V
11 ESP.motor_eff_fr
12 ESP.ESPPressureDegradation
13 ESP.ESPpump.RateDegradation
14 ESP.power_degr_fr
15 Разделитель данных
16 well.Pline_atma
17 well.Pwf_atma
18 well.Tline_C
19 well.Twh_C
20 well.Tbh_C
21 well.ESP.ESPpump.EffiencyESP_d
22 well.ESP.efficency_ESP_total_d
23 well.Pdis_atma
24 well.Pint_atma
25 well.Tint_C
26 well.ESP.Power_shaft_W / k
27 well.ESP.Power_motor_W / k
28 well.ESP.ESPpump.PowerFluid_Wt / k
29 well.ESP.cable_Power_W / k
30 well.ESP.Power_CS_calc_W / k
31 well.ESP.cable_dPower_W / k
32 well.ESP.dPower_GasSep_W / k
33 well.ESP.dPower_protector_W / k
34 well.ESP.dPower_transform_W / k
35 well.ESP.I_A
36 well.ESP.ESPpump.PowerESP_Wt / k
37 Unnamed: 37


Первичное построение графиков

In [11]:
trace1_name = needed_param_list[12]
trace1 = go.Scattergl(
    x = result_data.index,
    y = result_data[trace1_name],
    name = trace1_name,
    mode = 'lines'
)

trace2_name = needed_param_list[13]
trace2 = go.Scattergl(
    x = result_data.index,
    y = result_data[trace2_name],
    name = trace2_name,
    mode = 'lines'
)
trace3_name = needed_param_list[14]
trace3 = go.Scattergl(
    x = result_data.index,
    y = result_data[trace3_name],
    name = trace3_name,
    mode = 'lines'
)

trace4_name = needed_param_list[3]
trace4 = go.Scattergl(
    x = result_data.index,
    y = result_data[trace4_name] / 100,
    name = trace4_name + "/100",
    mode = 'lines'
)



data = [trace1, trace2, trace3, trace4]


layout = dict(title = 'Коэффициенты деградации от 28 до 2 февраля'
             )

fig = dict(data=data, layout=layout)

iplot(fig, filename='basic-scatter')